In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig

In [2]:
pipe_detect = pipeline('ner', model="akdeniz27/bert-base-turkish-cased-ner",aggregation_strategy="first",device="cuda")

pipe_generate = pipeline("text-generation", model="ai-forever/mGPT",device="cuda", 
                return_full_text=True,
                repetition_penalty=1.1)


In [3]:
import pandas as pd

splits = {'train': 'train.csv', 'test': 'test.csv'}
df = pd.read_csv("hf://datasets/winvoker/turkish-sentiment-analysis-dataset/" + splits["train"])

In [4]:
df = df[df["dataset"]=="wiki"].convert_dtypes(convert_string=True)
df_text = df["text"].to_list()

In [5]:
print(len(df_text))

153364


In [6]:
import torch
import torch.utils
import torch.utils.data
from msnet.model import DataSet
from tqdm.auto import tqdm
ds = DataSet(df_text,df_text)
dl=torch.utils.data.DataLoader(ds,batch_size=1)

In [7]:
print(len(dl))

153364


In [8]:
results = []

for i,(text,_) in tqdm(enumerate(dl)):
    if (i+1)==25000:
        break
    r = pipe_generate([*text],top_k=50,top_p=0.9,do_sample=True,min_length=32,truncation=True,temperature=0.3,max_new_tokens=32)

    results.append({"text":str(r[0][0]["generated_text"]).casefold(),"label":"Notr"})



0it [00:00, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [11]:
cleared_results = []
for row in results:
    row["text"] = " ".join(row["text"].split())
    cleared_results.append(row)

In [12]:
pd.DataFrame(cleared_results).to_csv("/home/musasina/projects/teknofest/msnet/datasets/data_notr.csv",index=False)

In [9]:
import pandas as pd
df = pd.read_csv("/home/musasina/projects/teknofest/msnet/datasets/countries.csv")["surnames"].convert_dtypes(convert_string=True).to_list()
result = []
for row in df:
    text = row.split()
    for country in text:
        result.append(str(country).casefold().strip())

pd.DataFrame(data=result,columns=["names"]).to_csv("/home/musasina/projects/teknofest/msnet/datasets/countries.csv",index=False)